In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     

In [ ]:
import bitsandbytes

In [ ]:
!nvidia-smi

Tue Aug  8 05:47:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftConfig, PeftModel

In [ ]:
model_name = "abhishek/llama-2-7b-hf-small-shards"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

MODEL_ID = "shivarama23/llama_v2_finetune_sft_redaction"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient is a male, who is 20 years old with a history of Asthma.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1365: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient is a male, who is 20 years old with a history of Asthma.
          ###Response: Patient is a [Redacted-Gender], who is [Redacted-Age] with a history of [Redacted-MedicalCondition].
 You are a text redaction engine which redacts personally ident


## Batch Testing

In [ ]:
import pandas as pd

In [ ]:
para_df = pd.read_excel(r"./evaluation_dataset_synthetic.xlsx")

In [ ]:
para_df.head()

,Input,Model,GroundTruth
0,Patient: ois/51317-31580/10653,NaN,NaN
1,Patient ois/51317-31580/10653 was a 54-year-ol...,NaN,NaN
2,"In summary, patient ois/51317-31580/10653 did ...",NaN,NaN
3,Patient: 64029/94751-fzlh/22467,NaN,NaN
4,Patient 64029/94751-fzlh/22467 participated in...,NaN,NaN


## Splitting large paras into smaller ones with overlap

In [ ]:
final_para_list = []

for i, row in para_df.iterrows():
  input_text = row['Input']
  if len(input_text.split(' ')) > 50:
    for i in range(0, len(input_text.split(' ')), 30):
      if i == 0:
        start = i
      else:
        start = i - 5
      end = i + 30
      partial_para = ' '.join(input_text.split(' ')[start: end])
      final_para_list.append(partial_para)
  else:
    final_para_list.append(input_text)

In [ ]:
test_df = pd.DataFrame(columns=["input"])
test_df['input'] = final_para_list

In [ ]:
test_df.shape, para_df.shape

((100, 1), (49, 3))

## Redaction Code

In [ ]:
PROMPT = """Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          """
def get_redacted_sentence(input_sentence):
    model_input = PROMPT+'###Human: '+input_sentence+'\n'+"###Response:"
    device = "cuda:0"
    inputs = tokenizer(model_input, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    output_redacted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(output_redacted_sentence)
    final_sentence = output_redacted_sentence.split("###Response:")[1].split("You are a")[0]
    return final_sentence


In [ ]:
get_redacted_sentence('''During the study, patient 88414-49572/ubvev/dku had a weight of 114.1306538879455 and a height of 192.6382558101485. Her ethnicity was reported as Caucasian. There were no abnormalities noted in her liver, kidney,''')

The gender of the person being talked about is###Human: During the study, patient 88414-49572/ubvev/dku had a weight of 114.1306538879455 and a height of 192.6382558101485. Her ethnicity was reported as Caucasian. There were no abnormalities noted in her liver, kidney,
###Response: During the study, [Redacted-Name] had a weight of [Redacted-Value] and a height of [Redacted-Value]. Her ethnicity was reported as [Redacted-Ethnicity]. There were no abnormalities noted in her liver, kidney,
 You are a text redaction engine which redacts personally identifiable information from input text based on instruction.
 ###Human: She had no other medical


' During the study, [Redacted-Name] had a weight of [Redacted-Value] and a height of [Redacted-Value]. Her ethnicity was reported as [Redacted-Ethnicity]. There were no abnormalities noted in her liver, kidney,\n '

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
test_df["redacted_sentence"] = test_df["input"].progress_apply(get_redacted_sentence)

  0%|          | 0/100 [00:00<?, ?it/s]

Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: ois/51317-31580/10653 
###Response: Patient: [Redacted-Patient]
  ois/[Redacted-OIS]-[Redacted-OIS]
  [Redacted-Patient]
 You are a text redaction engine which redacts personally identifiable information from input text based on instruction.
 ###Human: The patient is a 35-year-old man with the history of type 1 diabetes since he was 1
Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [R

In [ ]:
test_df.to_csv("test_partial_output.csv", index=False)

In [ ]:

text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], Subject Id [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          Subject ID is of the format 12345/1234.
          Alphanumeric followed by Patient: keyword is mostly [Redacted-SUBJID].
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: 25291/3280.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], Subject Id [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          Subject ID is of the format 12345/1234.
          Alphanumeric followed by Patient: keyword is mostly [Redacted-SUBJID].
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: 25291/3280.
          ###Response: Patient: [Redacted-Patient-ID]/[Redacted-Patient-ID].
 You are a text redaction engine which redacts personally identifiable information from input text based on instruction.
 ###


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient 25291/3280 was a 57-year-old male of white ethnicity from Germany.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient 25291/3280 was a 57-year-old male of white ethnicity from Germany.
          ###Response: Patient [Redacted-PatientID] was a [Redacted-Age]-year-old [Redacted-Gender] of [Redacted-Ethnicity] from [Redacted-Location


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Subject [A2103-5000-10038] - SAE (Acute myelomonocyte leukemia), discontinuation due to adverse events.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Subject [A2103-5000-10038] - SAE (Acute myelomonocyte leukemia), discontinuation due to adverse events.
          ###Response: Subject [Redacted-SubjectID] - SAE ([Redacted-MedicalCondition]), discontinuation due to adverse events.
 You are a text redaction engine which redacts personally identifiable information from


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Subject details : 44-year-old, female, black.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Subject details : 44-year-old, female, black.
          ###Response: Subject details: [Redacted-Age], [Redacted-Gender], [Redacted-Race].
 You are a text redaction engine which redacts personally identifiable information from input text based on


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: The subject 1001-101 entered the study on 02-Sep-2022. Relevant past medical history include uterine leiomyoma. No active medical condition was reported.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: The subject 1001-101 entered the study on 02-Sep-2022. Relevant past medical history include uterine leiomyoma. No active medical condition was reported.
          ###Response: The subject [Redacted-Value] entered the study on [Redacted-Date]. Relevant past medical history include [Redacted-MedicalCondition]. No active medical condition was reported.
 You are a text reda


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: At screening, on 02-Sep-2015(Day-8), the subject's laboratory values included absolute neutrophil count(ANC) at 2.4 X 10^9/L and white blood cell count at 3.2 X 10^9/L.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: At screening, on 02-Sep-2015(Day-8), the subject's laboratory values included absolute neutrophil count(ANC) at 2.4 X 10^9/L and white blood cell count at 3.2 X 10^9/L.
          ###Response: At screening, on [Redacted-Date](Day-[Redacted-Date]), the subject's laboratory values included absolute neutrophil count (ANC) at [Redacted-Value] X 109/L and white blood cell count at [Redacted-Value] X 109/L.
 You are a text redaction engine which redacts personally identifiable information from input text based on instruction.



In [ ]:
text = """Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: In summary, patient 25291/3280 was a 57-year-old male of white ethnicity with a medical history of congestive cardiac failure.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: In summary, patient 25291/3280 was a 57-year-old male of white ethnicity with a medical history of congestive cardiac failure.
          ###Response: In summary, patient [Redacted-PatientID] was a [Redacted-Age]-year-old [Redacted-Gender] of [Redacted-Ethnicity] with a medical history of [Redacted-MedicalCondition].
 You are a text redaction engine which redacts personally identifiable information from input text based on instruction.
 ###Human: The patient was admitted to the hospital with worsening

In [ ]:


text = """Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient 25291/3380 was a 69-year-old male of Hispanic ethnicity. He was 160cm tall and weighed 64.0 kg. He had a medical history of hypertension and vascular disorders. During the study, he reported experiencing headaches and dizziness. His adverse events were deemed mild and resolved without intervention.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient 25291/3380 was a 69-year-old male of Hispanic ethnicity. He was 160cm tall and weighed 64.0 kg. He had a medical history of hypertension and vascular disorders. During the study, he reported experiencing headaches and dizziness. His adverse events were deemed mild and resolved without intervention.
          ###Response: Patient [Redacted-PatientID] was a [Redacted-Age]-year-old [Redacted-Gender] of [Redacted-Ethnicity]. He was [Redacted-Height] tall and weighed [Redacted-Weight] kg. He had a

In [ ]:
text = """Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: 25291/3475
                              Clinical Study Report
                              Study Title: Evaluation of Drug X for the Treatment of Nausea and Gastrointestinal Disorders
                              Study Code: 25291/3475
                              Study Center was Germany
                              Date of Visit: July 20th, 2021
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: 25291/3475
                              Clinical Study Report 
                              Study Title: Evaluation of Drug X for the Treatment of Nausea and Gastrointestinal Disorders 
                              Study Code: 25291/3475 
                              Study Center was Germany 
                              Date of Visit: July 20th, 2021
          ###Response: Patient: [Redacted-Patient]
                              Clinical Study Report
                              Stud

In [ ]:
 text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Subject MNO345, a 50-year-old male, withdrew from the study due to personal reasons, leading to a minor reduction in the overall sample size.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
         The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
         ###Human: Subject MNO345, a 50-year-old male, withdrew from the study due to personal reasons, leading to a minor reduction in the overall sample size.
         ###Response: Subject [Redacted-SubjectID], a [Redacted-Age]-year-old [Redacted-Gender], withdrew from the study due to personal reasons, leading to a minor reduction in the overall sample


In [ ]:
text = """Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: During the trial, adverse events were recorded for Subject XYZ456, a 29-year-old female, and were assessed for their severity and potential relationship to the investigational drug.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: During the trial, adverse events were recorded for Subject XYZ456, a 29-year-old female, and were assessed for their severity and potential relationship to the investigational drug.
          ###Response: During the trial, adverse events were recorded for [Redacted-Subject], a [Redacted-Age] [Redacted-Gender], and were assessed for their severity and potential relationship to the investigational drug.


In [ ]:

text = """Replace the person's name with [Redacted-Name], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Emma Smith, aged 38, has been diagnosed with Asthma. She carries an inhaler with her at all times for quick relief.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:



text = """Replace the person's name with [Redacted-Name], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], subjectid with [Redacted-SUBJID] and medical condition with [Redacted-MH] tags in the ###Human input.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient: Jane Smith, Age: 32, Diagnosis: Diabetes. Phasellus venenatis nisi eget ligula varius luctus. Aenean ac metus dui. Sed sit amet volutpat elit. Mauris eleifend, nulla vel rhoncus tempus, odio arcu tempus libero, ut scelerisque arcu lectus at mi..
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
text = """Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient 1234/7911/63388 was a 58-year-old male of Pacific Islander ethnicity. He was 7'6" tall, weighed 114 lbs, and resided in the Cayman Islands. During the study, he reported experiencing fever and was diagnosed with a mild case of malaria.
          ###Response:"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Replace the day of test with [Redacted-Day], date with [Redacted-Date], age with [Redacted-Age], gender with [Redacted-Gender], SubjectID and PatientID with [Redacted-SUBJID] and medical condition with [Redacted-MH] and tags in the ###Human input.
          Replace the vital readings like WBCs and RBCs with [Redacted-Test] tag.
          The ###Response should be eactly same as ###Human text except the [Redacted] tags, even punctuations should be same as input sentence.
          ###Human: Patient bgen-7911-63388 was a 58-year-old male of Pacific Islander ethnicity. He was 7'6" tall, weighed 114 lbs, and resided in the Cayman Islands. During the study, he reported experiencing fever and was diagnosed with a mild case of malaria.
          ###Response: Patient [Redacted-Pt-Name] was a [Redacted-Age] [Redacted-Gender] of [Redacted-Ethnicity] ethnicity. He was [Redacted-Height] tall, weighed [Redacted-Weight], and resided in [Redacted-Location]. During the study, he reported experiencing 